# PicCompare - Duplicate Image Finder Plan

*Note:* This application is specifically designed and planned for the **Kubuntu Linux distribution** (using KDE Plasma/Qt). The choice of PySide6 for the GUI aligns well with this target environment.

## 1. Proposed Project Structure

```
PicCompare/
├── main.py             # Entry point, initializes the application
├── core/
│   ├── __init__.py
│   ├── scanner.py        # Functions for finding image files
│   ├── hasher.py         # Functions for calculating image hashes (using ImageHash & Pillow)
│   ├── duplicate_finder.py # Logic to group images by hash and identify duplicates
│   └── file_handler.py   # Functions for deleting files (using send2trash)
├── ui/
│   ├── __init__.py
│   ├── main_window.py    # Main application window class (PySide6)
│   ├── widgets/
│   │   ├── __init__.py
│   │   ├── directory_selector.py # Widget for selecting directories
│   │   ├── progress_display.py   # Widget for showing progress
│   │   ├── duplicate_list.py     # Widget to list duplicate sets
│   │   └── image_compare.py      # Widget for side-by-side comparison and selection
│   └── resources/          # Optional: Icons, etc.
├── requirements.txt    # Lists project dependencies (Pillow, ImageHash, PySide6, send2trash)
└── README.md           # Project description, setup, usage
```

## 2. Core Logic Details (`core/` modules)

*   **`scanner.py`:**
    *   `find_image_files(directory_paths: list[str]) -> list[str]`: Takes a list of directories, recursively scans them, and returns a list of paths to files with supported extensions (.jpg, .jpeg, .png, .gif). Uses `os.walk` or `pathlib.rglob`.
*   **`hasher.py`:**
    *   `calculate_perceptual_hash(image_path: str) -> str | None`: Takes an image path, opens it with `Pillow`, calculates its dHash using `ImageHash`, and returns the hash string. Handles potential errors during image opening/processing.
*   **`duplicate_finder.py`:**
    *   `group_by_hash(image_paths: list[str]) -> dict[str, list[str]]`: Takes the list of image paths, calculates the hash for each (using `hasher.calculate_perceptual_hash`), and returns a dictionary mapping hashes to lists of file paths.
    *   `identify_duplicates(hash_groups: dict[str, list[str]]) -> list[list[str]]`: Takes the hash groups dictionary and returns a list where each inner list contains the file paths of images considered duplicates (i.e., groups with more than one path for the same hash).
*   **`file_handler.py`:**
    *   `move_to_trash(file_paths: list[str]) -> None`: Takes a list of file paths and moves them to the system's trash/recycle bin using the `send2trash` library. This is safer than permanent deletion initially.

## 3. UI Details (`ui/` modules using PySide6)

*   **`main_window.py` (`MainWindow` class):**
    *   Main application layout (e.g., `QVBoxLayout`, `QHBoxLayout`).
    *   Integrates the custom widgets: `DirectorySelector`, `ProgressDisplay`, `DuplicateList`, `ImageCompare`.
    *   Connects signals/slots:
        *   "Scan" button triggers directory scanning, hashing, and duplicate finding (using `core` functions). Updates `ProgressDisplay` and `DuplicateList`.
        *   Selecting a set in `DuplicateList` updates the `ImageCompare` widget.
        *   "Delete Selected" button in `ImageCompare` triggers `file_handler.move_to_trash` after confirmation. Updates `DuplicateList` and `ImageCompare`.
*   **`widgets/directory_selector.py`:**
    *   Button to open `QFileDialog.getExistingDirectory` multiple times or a list view to manage selected paths.
    *   Provides a signal `directories_selected(list[str])`.
*   **`widgets/progress_display.py`:**
    *   A `QProgressBar` and potentially a `QLabel` for status text.
    *   Methods like `update_progress(value, max_value, status_text)`.
*   **`widgets/duplicate_list.py`:**
    *   A `QListWidget` or `QTreeView` to display the identified duplicate sets (e.g., showing the hash or a representative thumbnail and the number of files).
    *   Provides a signal `set_selected(list[str])` emitting the file paths for the chosen set.
*   **`widgets/image_compare.py`:**
    *   Layout (e.g., `QHBoxLayout` or `QGridLayout`) to display multiple images.
    *   Uses `QLabel` with `QPixmap` to show images (scaled appropriately).
    *   Adds `QCheckBox` under/next to each image for selection (Keep/Delete). Logic to ensure at least one is kept? Or maybe just select for deletion. Let's default to "select for deletion".
    *   Navigation buttons ("Next Set", "Previous Set") if displaying one set at a time.
    *   "Delete Selected Images in Set" button.
    *   Provides a signal `delete_requested(list[str])` with paths to delete.


## 4. `main.py` Outline

```python
import sys
from PySide6.QtWidgets import QApplication
from ui.main_window import MainWindow

if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = MainWindow()
    window.show()
    sys.exit(app.exec())
```

## 5. Dependencies (`requirements.txt`)

```
Pillow>=9.0.0
ImageHash>=4.2.0
PySide6>=6.3.0
send2trash>=1.8.0
```

## 6. Mermaid Diagram (Simplified UI Layout Concept)

```mermaid
graph TD
    subgraph MainWindow
        direction TB
        A[Directory Selector Widget] --> B(Scan Button);
        B --> C{Core Logic};
        C --> D[Progress Display Widget];
        C --> E[Duplicate List Widget];
        E -- Select Set --> F[Image Compare Widget];
        F -- Request Deletion --> G(Confirm Dialog);
        G -- Confirmed --> H{Core Logic - Delete};
        H --> E; # Update List
        H --> F; # Update/Clear View
    end

    style C fill:#f9f,stroke:#333,stroke-width:2px
    style H fill:#f9f,stroke:#333,stroke-width:2px
```